In [ ]:
%matplotlib notebook

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

from utils.plot import graph_w_widgets
from utils.math import sampling_times, superposed_waves, wave, whittaker_shannon

# set the default figure size for plots
plt.rcParams["figure.figsize"] = (8, 4.5)

# some constants
TIME_0 = 0
TIME_1 = 10
N = 5000
TIME = np.linspace(TIME_0, TIME_1, N)

# How can we describe a sound wave?

$$\Large a(t) = A\sin(2 \pi f t)$$

where:

- $a(t)$ is the amplitude in a given moment of time $t$
- $A$ is the maximum amplitude of the wave
- $f$ is the frequency of the wave
- $t$ is a given moment in time

In [ ]:
def on_update(fig, lines, amp, freq):
    lines[0].set_ydata(wave(TIME, amp, freq))
    fig.canvas.draw_idle()

graph_w_widgets(
    graph={
        "lines": [
            {"x": TIME, "y": wave(TIME, 5, 0.5), "color": "blue", "lw": 2},
        ],
        "x_label": "Time [s]",
        "y_label": "Amplitude",
        "y_lims": [-6, 6],
    },
    widgets=[
        {
            "amp": widgets.FloatSlider(
                value=5,
                min=0,
                max=5,
                step=0.5,
                orientation="vertical",
                description=r"$A$",
            ),
        },
        {
            "freq": widgets.FloatSlider(
                value=0.5,
                min=0,
                max=5,
                step=0.5,
                orientation="vertical",
                description=r"$f$",
            ),
        },
    ],
    on_update=on_update,
)

# Wave superposition

The principle of superposition can be applied to 2 or more waves travelling through the same medium at the same time. Each wave passes without disturbing the others, so the net displacement is given by:

$$\Large a(t) = A_1\sin(2 \pi f_1 t) + A_2\sin(2 \pi f_2 t) + A_3\sin(2 \pi f_3 t) + \ldots$$

In [ ]:
def parse(waves):
    lines = waves.split("\n")
    result = []
    for line in lines:
        a, f = line.split(",")
        result.append((float(a), float(f)))
    
    return result

def on_update(fig, lines, waves):
    lines[0].set_ydata(superposed_waves(TIME, parse(waves)))
    fig.canvas.draw_idle()

graph_w_widgets(
    graph={
        "lines": [
            {"x": TIME, "y": superposed_waves(TIME, [(1, 1)]), "color": "blue", "lw": 2},
        ],
        "x_label": "Time [s]",
        "y_label": "Amplitude",
        "y_lims": [-6, 6],
    },
    widgets=[
        {
            "waves": widgets.Textarea(
                value="1, 1\n",
                placeholder="Enter some waves parameters",
                description="Waves",
                layout=widgets.Layout(width="145px", height="150px")
            ),
        },
    ],
    on_update=on_update,
)

# Sampling

Sound is a continuos signal, but that does not work well in the digital world, so it needs to be converted into a discrete signal. Using a process known as sampling, we split the signal into very small chunks and then perform something called quantization.

# Nyquist-Shannon theorem

> If a function $a(t)$ contains no frequencies higher than $F$ hertz, it is completely determined by giving its ordinates at a series of points spaced $1 / (2F)$ or seconds apart.

This theorem can be interpreted in two ways:

- Having $2F$ samples per second is enough to sample a continous signal with frequency $F$
- A sample rate $f_s$ guarantees perfect reconstruction for continous signals with $f < f_s / 2$

In [ ]:
def build_wave(time, use_superposition):
    if use_superposition:
        return superposed_waves(time, [(1, 1), (0.8, 1.7), (0.5, 2)])
    
    return wave(time, 1, 1)

def on_update(fig, lines, sampling_freq, show_rec, use_superposition):
    s_times = sampling_times(TIME_0, TIME_1, sampling_freq)
    samples = build_wave(s_times, use_superposition)
    
    lines[0].set_ydata(build_wave(TIME, use_superposition))
    lines[1].set_ydata(whittaker_shannon(TIME, samples, s_times))
    lines[2].set_xdata(s_times)
    lines[2].set_ydata(samples)
    
    if show_rec:
        lines[1].set_linestyle("-")
        lines[2].set_color("black")
    else:
        lines[1].set_linestyle("")
        lines[2].set_color("red")
        
    fig.canvas.draw_idle()
    
s_times = sampling_times(TIME_0, TIME_1, 0.1)
samples = build_wave(s_times, False)
graph_w_widgets(
    graph={
        "lines": [
            {"x": TIME, "y": build_wave(TIME, False), "color": "gray", "lw": 2},
            {"x": TIME, "y": whittaker_shannon(TIME, samples, s_times), "color": "red", "linestyle": "", "lw": "2"},
            {"x": s_times, "y": samples, "color": "red", "linestyle": "", "marker": "o"},
        ],
        "x_label": "Time [s]",
        "y_label": "Amplitude",
        "y_lims": [-2.5, 2.5],
    },
    widgets=[
        {
            "use_superposition": widgets.Checkbox(
                value=False,
                description="Superposition",
                disabled=False,
                indent=False,
                layout=widgets.Layout(width="145px")
            ),
            "show_rec": widgets.Checkbox(
                value=False,
                description="Show reconstruction",
                disabled=False,
                indent=False,
                layout=widgets.Layout(width="145px")
            ),
            "sampling_freq": widgets.FloatSlider(
                min=0.1,
                max=15,
                step=0.1,
                orientation="vertical",
                description="Sampling frequency",
            ),
        },
    ],
    on_update=on_update,
)